# Algoritmo Genético con fase de Mejora Local, se implementan cruces con 1, 3, 4 y 5 puntos de cruce. También seimplementa una heuristica que plantea que cada 100 generaciones: si el mejor hijo no supera al mejor padre, se cambia el metodo de cruce como una estrategia para escapar de minimos locales.


Colab: https://drive.google.com/file/d/1sJxcTEUr6AYkJB0vE6ZzerbFkrRZTmPN/view?usp=sharing

GitHub: https://github.com/kmotillo/03MAIR_ALGORITMOS_OPTIMIZACION/blob/main/AG3/TSP_GENETICO.ipynb

# Cargamos todas la librerias

In [ ]:
#!pip install tsplib95
#!pip install pandas

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import random
from time import time
import copy
import urllib.request #Hacer llamadas http a paginas de la red
import tsplib95       #Modulo para las instancias del problema del TSP
import math           #Modulo de funciones matematicas. Se usa para exp


# Descargamos el fichero de datos(Matriz de distancias)
### Carga de datos y generación de objeto problem

In [ ]:

#http://elib.zib.de/pub/mp-testdata/tsp/tsplib/
#Documentacion : 
  # http://comopt.ifi.uni-heidelberg.de/software/TSPLIB95/tsp95.pdf
  # https://tsplib95.readthedocs.io/en/stable/pages/usage.html
  # https://tsplib95.readthedocs.io/en/v0.6.1/modules.html
  # https://pypi.org/project/tsplib95/

#file = "swiss42.tsp"
#file = "fri26.tsp" ;
#file = "dantzig42.tsp" ;

def loadProblem(file):
    url = "http://elib.zib.de/pub/mp-testdata/tsp/tsplib/tsp/"+file+""
    urllib.request.urlretrieve(url, file) 

    # Aquí Cargamos el problema en la variable problem
    problem = tsplib95.load(file)

    #Nodos
    Nodos = list(problem.get_nodes())

    #Aristas
    Aristas = list(problem.get_edges())

    #Distancia entre nodos
    #problem.get_weight(1, 1)

    n = problem.dimension

    #Obtengo los pesos mas pequeños

    return n,problem,Nodos

# Funcion de Fitness Viajante de Comercio (TS)

In [ ]:
def funcionDeCalidad(individuo, problem):
    foSum = 0
    for i in range(0, len(individuo)-1):
        foSum+=problem.get_weight(individuo[i] ,individuo[i+1])
    foSum+=problem.get_weight(individuo[len(individuo)-1], individuo[0])
    return foSum

# Poblacion Inicial
Aquí la idea es Construir un conjunto de soluciónes aleatoreas para asegurar la diversidad de los indivuidos.

In [ ]:
def poblacion_inicial(n,N,Nodos): 
    poblacion = []
    for i in range(N):
        solucion = [random.choice(Nodos)]
        for j in range(n-1):
            not_in_Solution = list(set(Nodos) - set(solucion))
            solucion = solucion + [random.choice(not_in_Solution)]
        poblacion.append(solucion)
    
    return poblacion 

# Fase de Seleccion

In [ ]:
def fitness_prob(poblacionInicial):
    fitness =[]
    for individuo in poblacionInicial:
        fitness += [funcionDeCalidad(individuo, problem)]
        
    fitness = np.array(fitness)
    fitnessProb=fitness/fitness.sum()
    return fitnessProb

def seleccion(poblacionInicial):
    fitnessProb = fitness_prob(poblacionInicial)
    size_poblacion = len(poblacionInicial)
    while True:
        parents = np.random.choice(size_poblacion, 2, p=fitnessProb)
        if parents[0]!=parents[1]: break
    return poblacionInicial[parents[0]], poblacionInicial[parents[1]]

# Fase de Cruce

In [ ]:
def one_point_cross_over(padres, problem, mut_fact):
    Nodos = list(problem.get_nodes())
    #Se elige un punto de corte aleatorio:
    pc = random.sample( range(1,len(padres[0])),1)[0]
    hijo1 =  comprobar_hijo(padres[0][:pc] + padres[1][pc:] ,problem) 
    hijo2 =  comprobar_hijo(padres[1][:pc] + padres[0][pc:] ,problem)
    #return [mutacion_extrema(hijo1,mut_fact),mutacion_extrema(hijo2,mut_fact)]
    #return [mutacion_extrema(hijo1,mut_fact),mutacion_extrema(hijo2,mut_fact)]
    return [local_search_FI_swap(mutacion_extrema(hijo1,mut_fact),problem), local_search_2_opt(mutacion_extrema(hijo2,mut_fact), problem)]

def three_point_cross_over(padres, problem, mut_fact, k=3):
    Nodos = list(problem.get_nodes())
    #Se elige un punto de corte aleatorio:
    while True:
        pc = sorted(random.sample(range(3,len(poblacion[0])-3),k))
        if(pc[2]-pc[1]) > 7 and (pc[1]-pc[0]) > 7:
            break                                    
    hijo1 =  comprobar_hijo(padres[0][:pc[0]] + padres[1][pc[0]:pc[1]] + padres[0][pc[1]:pc[2]] + padres[1][pc[2]:] ,problem) 
    hijo2 =  comprobar_hijo(padres[1][:pc[0]] + padres[0][pc[0]:pc[1]] + padres[1][pc[1]:pc[2]] + padres[0][pc[2]:] ,problem)
    return [mutacion_extrema(hijo1,mut_fact), mutacion_extrema(hijo2,mut_fact)]
    #return [local_search_2_opt(mutacion(hijo1,mut_fact),problem),local_search_2_opt(mutacion(hijo2,mut_fact),problem)]
    
def four_point_cross_over(padres, problem, mut_fact, k=4):
    Nodos = list(problem.get_nodes())
    #Se elige un punto de corte aleatorio:
    while True:
        pc = sorted(random.sample(range(3,len(poblacion[0])-3),k))
        if (pc[3]-pc[2]) > 4 and (pc[2]-pc[1]) > 4 and (pc[1]-pc[0]) > 4:
            break                                    
    hijo1 =  comprobar_hijo(padres[0][:pc[0]] + padres[1][pc[0]:pc[1]] + padres[0][pc[1]:pc[2]] + padres[1][pc[2]:pc[3]] + padres[0][pc[3]:] ,problem) 
    hijo2 =  comprobar_hijo(padres[1][:pc[0]] + padres[0][pc[0]:pc[1]] + padres[1][pc[1]:pc[2]] + padres[0][pc[2]:pc[3]] + padres[1][pc[3]:] ,problem)
    #return [mutacion_extrema(hijo1,mut_fact), mutacion_extrema(hijo2,mut_fact)]
    return [local_search_BI_swap(mutacion_extrema(hijo1,mut_fact),problem), local_search_2_opt(mutacion_extrema(hijo2,mut_fact), problem)]

def five_point_cross_over(padres, problem, mut_fact, k=5):
    Nodos = list(problem.get_nodes())
    #Se elige un punto de corte aleatorio:
    while True:
        pc = sorted(random.sample(range(3,len(poblacion[0])-3),k))
        if (pc[4]-pc[3]) > 4 and (pc[3]-pc[2]) > 4 and (pc[2]-pc[1]) > 4 and (pc[1]-pc[0]) > 4:
            break                                    
    hijo1 =  comprobar_hijo(padres[0][:pc[0]] + padres[1][pc[0]:pc[1]] + padres[0][pc[1]:pc[2]] + padres[1][pc[2]:pc[3]] + padres[0][pc[3]:pc[4]] + padres[1][pc[4]:],problem) 
    hijo2 =  comprobar_hijo(padres[1][:pc[0]] + padres[0][pc[0]:pc[1]] + padres[1][pc[1]:pc[2]] + padres[0][pc[2]:pc[3]] + padres[1][pc[3]:pc[4]] + padres[0][pc[4]:],problem)
    #return [mutacion_extrema(hijo1,mut_fact), mutacion_extrema(hijo2,mut_fact)]
    return [local_search_FI_swap(mutacion_extrema(hijo1,mut_fact),problem), local_search_BI_swap(mutacion_extrema(hijo2,mut_fact), problem)]


def k_point_cross_over_elit(padres, problem, mut_fact, k=3):
    Nodos = list(problem.get_nodes())
    #Se elige un punto de corte aleatorio:
    while True:
        pc = sorted(random.sample(range(3,len(poblacion[0])-3),k))
        if(pc[2]-pc[1]) > 3 and (pc[1]-pc[0]) > 3:
            break                                    
    hijo1 =  comprobar_hijo(padres[0][:pc[0]] + padres[1][pc[0]:pc[1]] + padres[0][pc[1]:pc[2]] + padres[1][pc[2]:] ,problem) 
    hijo2 =  comprobar_hijo(padres[1][:pc[0]] + padres[0][pc[0]:pc[1]] + padres[1][pc[1]:pc[2]] + padres[0][pc[2]:] ,problem)
    #return [local_search_BI_swap(local_search_2_opt(mutacion_extrema(hijo1,mut_fact),problem),problem),local_search_BI_swap(local_search_2_opt(mutacion_extrema(hijo2,mut_fact), problem), problem)]
    return [local_search_2_opt(mutacion(hijo1,mut_fact),problem),local_search_FI_swap(mutacion(hijo2,mut_fact),problem)]


def comprobar_hijo(hijo,problem):
    Nodos = list(problem.get_nodes())
    not_in_solution = list(set(Nodos) - set(hijo))
      #Recorremos todos los genes y cuandlo encontremos 
    for i in range(len(hijo) ):
        if hijo[i] in hijo[:i]:
            hijo[i] = not_in_solution.pop(0) #Cambiamos el gen y lo eliminamos de la lista not_in_solution
    return hijo 

def cross_over_4(poblacion,problem,mut_fact):
    #Definimos en una variable la copia de la población para ir eliminando los padres seleccionados
    poblacion_copia = copy.deepcopy(poblacion)

    #Definimos en una variable la copia de la población para ir añadiendo los hijos creados
    poblacion_final = copy.deepcopy(poblacion)

    while len(poblacion_copia) > 1:  #Iteramos mientras haya padres disponibles
        #Seleccionamos dos padres
        padre1,padre2 = random.sample(poblacion_copia, 2)
        #hijo1,hijo2 = k_point_cross_over([padre1,padre2],problem,mut_fact)
        hijo1,hijo2 = four_point_cross_over([padre1,padre2],problem,mut_fact)
        if (hijo1 not in poblacion_final and  hijo2 not in poblacion_final):
            poblacion_copia.remove(padre1)
            poblacion_copia.remove(padre2)
            poblacion_final.extend([hijo1,hijo2])
    return poblacion_final

def cross_over_5(poblacion,problem,mut_fact):
    #Definimos en una variable la copia de la población para ir eliminando los padres seleccionados
    poblacion_copia = copy.deepcopy(poblacion)

    #Definimos en una variable la copia de la población para ir añadiendo los hijos creados
    poblacion_final = copy.deepcopy(poblacion)

    while len(poblacion_copia) > 1:  #Iteramos mientras haya padres disponibles
        #Seleccionamos dos padres
        padre1,padre2 = random.sample(poblacion_copia   ,2)
        #padre1,padre2 = seleccion(poblacion_copia)
        hijo1,hijo2 = five_point_cross_over([padre1,padre2],problem,mut_fact)
        if (hijo1 not in poblacion_final and  hijo2 not in poblacion_final):
            poblacion_copia.remove(padre1)
            poblacion_copia.remove(padre2)
            poblacion_final.extend([hijo1,hijo2])
    return poblacion_final

def cross_over_3(poblacion,problem,mut_fact):
    #Definimos en una variable la copia de la población para ir eliminando los padres seleccionados
    poblacion_copia = copy.deepcopy(poblacion)

    #Definimos en una variable la copia de la población para ir añadiendo los hijos creados
    poblacion_final = copy.deepcopy(poblacion)

    while len(poblacion_copia) > 1:  #Iteramos mientras haya padres disponibles
        #Seleccionamos dos padres
        padre1,padre2 = random.sample(poblacion_copia   ,2)
        hijo1,hijo2 = three_point_cross_over([padre1,padre2],problem,mut_fact)
        if (hijo1 not in poblacion_final and  hijo2 not in poblacion_final):
            poblacion_copia.remove(padre1)
            poblacion_copia.remove(padre2)
            poblacion_final.extend([hijo1,hijo2])
    return poblacion_final

def cross_over_1(poblacion,problem,mut_fact):
    #Definimos en una variable la copia de la población para ir eliminando los padres seleccionados
    poblacion_copia = copy.deepcopy(poblacion)

    #Definimos en una variable la copia de la población para ir añadiendo los hijos creados
    poblacion_final = copy.deepcopy(poblacion)

    while len(poblacion_copia) > 1:  #Iteramos mientras haya padres disponibles
        #Seleccionamos dos padres
        padre1,padre2 = random.sample(poblacion_copia   ,2)
        hijo1,hijo2 = one_point_cross_over([padre1,padre2],problem,mut_fact)
        if (hijo1 not in poblacion_final and  hijo2 not in poblacion_final):
            poblacion_copia.remove(padre1)
            poblacion_copia.remove(padre2)
            poblacion_final.extend([hijo1,hijo2])
    return poblacion_final

# Fase de Mutación

In [ ]:
def mutacion(hijo, mut_fact):
    if random.random() < mut_fact:
        gen1,gen2 = sorted(random.sample(range(len(hijo)),2))
        aux = hijo[gen1]
        hijo[gen1] = hijo[gen2]
        hijo[gen2] = aux
        return hijo
    else:
        return hijo[::] 
    
def mutacion_extrema(hijo, mut_fact):
    for i in range(len(hijo)):
        if random.random() <= mut_fact:
            gen = random.choice(list(set(range(len(hijo)))-{i}))
            aux = hijo[i]
            hijo[i] = hijo[gen]
            hijo[gen] = aux
    return hijo     

# Fase de reemplazo

In [ ]:
def reemplazo(poblacion, N, problem, fact_elit):
  #Se ordena la población según el fitness(tamaño del recorrido) en una lista de elementos [distancia, solucion]
  poblacion_ordenada = sorted([ [funcionDeCalidad(individuo, problem), individuo] for individuo in poblacion ], key= lambda x:x[0] )

  #Devolvemos elitismo% y el resto se eligen aleatoriamente
  return [x[1] for x in poblacion_ordenada][:int(N*fact_elit)]  + \
  random.sample([x[1] for x in poblacion_ordenada][int(N*fact_elit):] , int(N*(1-fact_elit))  ) 


#Evalua la población y devuelve el mejor individuo
def mejor_individuo(poblacion, problem):
    mejor_solucion = []
    fitness = math.inf
    for individuo in poblacion:
        _fitness = funcionDeCalidad(individuo, problem)
        if _fitness < fitness:
            mejor_solucion = individuo
            fitness = _fitness
    return mejor_solucion, fitness   

# Fase de Mejora Local

In [ ]:
def local_search_2_opt(solucion, problem):
    fo = funcionDeCalidad(solucion, problem)
    solLocal = copy.deepcopy(solucion)
    # notInSol = notInSol.difference(solInt)
    mejora = True
    # solucion = solInt
    saltos=0
    n = problem.dimension

    while (mejora):
        mejora = False
        
        for i in range(0, n-1):
            mejora = False
            elem1 = solLocal[i]
            elem2 = solLocal[i+1]

            solLocal[i] = elem2
            solLocal[i+1] = elem1

            _fo = funcionDeCalidad(solLocal, problem)
          #print("Sol:",i,solLocal,"FO:", _fo)
            if _fo < fo:
                #print(_fo)
                fo = _fo
                mejora = True 
            else:
                solLocal[i] = elem1
                solLocal[i+1] = elem2
        saltos+=1
        
    return solLocal
def local_search_FI_swap(solucion, problem):
    fo = funcionDeCalidad(solucion, problem)
    solLocal = copy.deepcopy(solucion)
    # notInSol = notInSol.difference(solInt)
    mejora = True
    # solucion = solInt
    saltos=0
    n = problem.dimension

    while (mejora):
        mejora = False
        
        for i in range(0, n):
            mejora = False
            elem1 = solLocal[i]
            for j in range(i+1, n):
                elem2 = solLocal[j]
                solLocal[i] = elem2
                solLocal[j] = elem1
                _fo = funcionDeCalidad(solLocal, problem)
              
                #print("Sol:",i,j,solLocal,"FO:", _fo)
                if _fo < fo:
                    #print(_fo)
                    fo = _fo
                    mejora = True 
                    break
                else:
                    solLocal[i] = elem1
                    solLocal[j] = elem2
        saltos+=1
        
    return solLocal

def local_search_BI_swap(solucion, problem):
    fo = funcionDeCalidad(solucion, problem)
    solLocal = copy.deepcopy(solucion)
    # notInSol = notInSol.difference(solInt)
    mejora = True
    # solucion = solInt
    saltos=0
    n = problem.dimension
    mejorSolTemp = copy.deepcopy(solucion)

    while (mejora):
        mejora = False
        
        for i in range(0, n):
            mejora = False
            elem1 = solLocal[i]
            fo = funcionDeCalidad(solLocal, problem)
            for j in range(i+1, n):
                elem2 = solLocal[j]
                solLocal[i] = elem2
                solLocal[j] = elem1
                _fo = funcionDeCalidad(solLocal, problem)
              
                #print("Sol:",i,j,solLocal,"FO:", _fo)
                if _fo < fo:
                    #print(_fo)
                    fo = _fo
                    mejorSolTemp = copy.deepcopy(solLocal)
                    mejora = True

                solLocal[i] = elem1
                solLocal[j] = elem2
            
                
        saltos+=1
        
    return mejorSolTemp


# El presente algoritmo Genetico es una aproximacion a El algoritmo Chu-Beasley, decimos aproximación porque no se implementa la fase de aspiracion.

In [ ]:
%%time
file = "swiss42.tsp"
N = 500
n,problem,Nodos = loadProblem(file)
poblacion = poblacion_inicial(n,N,Nodos)
fact_elit = .40
mut_fact = .08
generaciones =1000
fo = np.inf

parar = False
cont=0
no_mejora_3 = 0
no_mejora_4 = 0
no_mejora_5 = 0
flag_3 = True
flag_4 = False
flag_5 = False

tampoco_mejora = 0
#Inciamos el cliclo de generaciones
flag = True
while(generaciones > cont) :
    cont+=1
    #Cruce de la poblacion(incluye selección y mutación)
    #poblacion = cross_over_elit(poblacion,problem,mut_fact)
    if flag_3:
        poblacion = cross_over_1(poblacion,problem,mut_fact)
        poblacion = reemplazo(poblacion, N, problem, fact_elit)
        (mejor_ind, fitness) = mejor_individuo(poblacion, problem)
        print("Generacion #", cont, " Fase 3_cross_over \nLa mejor solución es:" , mejor_ind, "\ncon Fitness " , fitness, "\n")
    if flag_4:
        poblacion = cross_over_4(poblacion,problem,mut_fact)
        poblacion = reemplazo(poblacion, N, problem, fact_elit)
        (mejor_ind, fitness) = mejor_individuo(poblacion, problem)
        print("Generacion #", cont, " Fase 4_cross_over \nLa mejor solución es:" , mejor_ind, "\ncon Fitness " , fitness, "\n")
    if flag_5:
        poblacion = cross_over_5(poblacion,problem,mut_fact)
        poblacion = reemplazo(poblacion, N, problem, fact_elit)
        (mejor_ind, fitness) = mejor_individuo(poblacion, problem)
        print("Generacion #", cont, " Fase 5_cross_over \nLa mejor solución es:" , mejor_ind, "\ncon Fitness " , fitness, "\n")
    
#     poblacion = reemplazo(poblacion, N, problem, fact_elit)
#     (mejor_ind, fitness) = mejor_individuo(poblacion, problem)
#     print("Generacion #", cont, "\nLa mejor solución es:" , mejor_ind, "\ncon Fitness " , fitness, "\n")
        
    if fitness < fo:
        fo = fitness
        no_mejora_3 = 0
        no_mejora_4 = 0
        no_mejora_5 = 0
    elif flag_3: 
        no_mejora_3+=1
        if no_mejora_3 >= 100:
            flag_3 = False
            flag_4 = True
            no_mejora_3 = 0
    elif flag_4:
        no_mejora_4+=1
        if no_mejora_4 >= 100:
            flag_4 = False
            flag_5 = True
            no_mejora_4 = 0
    else:
        no_mejora_5+=1
        if no_mejora_5 >= 100:
            flag_5 = False
            flag_3 = True
            no_mejora_5 = 0


Mejor Solucion encontrada para esta instancia :
Sol = [16, 15, 37, 17, 7, 1, 27, 28, 29, 9, 39, 21, 40, 24, 22, 38, 30, 32, 31, 36, 35, 20, 33, 34, 0, 6, 4, 3, 2, 8, 23, 41, 10, 25, 11, 12, 18, 26, 5, 13, 19, 14] 

con Fitness  1510

La mejor solución es: [0, 17, 36, 35, 31, 30, 29, 9, 41, 23, 40, 24, 21, 39, 22, 38, 34, 33, 20, 32, 2, 3, 4, 6, 1, 7, 37, 15, 16, 14, 19, 13, 5, 26, 18, 12, 11, 25, 10, 8, 28, 27] 
con Fitness  1493

La mejor solución es: [36, 35, 20, 33, 34, 32, 27, 2, 29, 39, 21, 40, 24, 22, 38, 30, 28, 0, 1, 7, 19, 13, 5, 6, 4, 3, 8, 9, 23, 41, 10, 25, 11, 12, 18, 26, 14, 16, 15, 37, 17, 31] 
con Fitness  1497 

La mejor solución es: [22, 39, 21, 40, 24, 38, 32, 34, 33, 20, 35, 36, 31, 17, 7, 5, 26, 18, 12, 10, 8, 9, 23, 41, 25, 11, 13, 19, 14, 16, 15, 37, 0, 1, 6, 4, 3, 27, 2, 28, 29, 30] 
con Fitness  1476 

La mejor solución es: [7, 17, 37, 15, 16, 14, 6, 1, 0, 31, 36, 35, 20, 33, 34, 32, 27, 3, 4, 2, 28, 8, 9, 29, 30, 38, 22, 39, 21, 24, 40, 23, 41, 10, 25, 11, 12, 18, 26, 5, 13, 19] 
con Fitness  1450 

La mejor solución es: [29, 28, 31, 17, 7, 19, 14, 16, 15, 37, 36, 35, 20, 33, 34, 32, 0, 1, 3, 27, 2, 4, 6, 5, 13, 26, 18, 12, 11, 25, 10, 8, 41, 23, 9, 21, 40, 24, 39, 22, 38, 30] 
con Fitness  1436 

La mejor solución es: [30, 29, 8, 41, 23, 9, 21, 40, 24, 39, 22, 38, 32, 34, 33, 20, 35, 36, 31, 17, 37, 15, 16, 14, 19, 13, 7, 1, 0, 27, 3, 4, 6, 5, 26, 18, 12, 11, 25, 10, 2, 28] 
con Fitness  1400

La mejor solución es: [33, 20, 35, 36, 17, 31, 32, 38, 22, 39, 24, 40, 21, 9, 23, 41, 8, 10, 25, 11, 12, 18, 26, 4, 6, 5, 13, 19, 14, 16, 15, 37, 7, 1, 0, 3, 2, 27, 28, 29, 30, 34] 
con Fitness  1384 

La mejor solución es: [0, 32, 20, 33, 34, 38, 22, 39, 24, 40, 21, 9, 23, 41, 4, 6, 1, 7, 17, 31, 35, 36, 37, 15, 16, 14, 19, 13, 5, 26, 18, 12, 11, 25, 10, 8, 29, 30, 28, 27, 2, 3] 
con Fitness  1384 

La mejor solución es: [21, 40, 24, 39, 22, 38, 30, 29, 28, 32, 34, 33, 20, 35, 36, 31, 17, 1, 6, 5, 13, 19, 14, 16, 15, 37, 7, 0, 27, 2, 3, 4, 26, 18, 12, 11, 25, 10, 8, 41, 23, 9] 
con Fitness  1361 

La mejor solución es: [19, 14, 16, 15, 37, 17, 31, 36, 35, 20, 33, 34, 32, 3, 4, 2, 27, 28, 29, 30, 38, 22, 39, 24, 40, 21, 9, 23, 41, 8, 10, 25, 11, 12, 18, 26, 6, 1, 0, 7, 5, 13] 
con Fitness  1329 

La mejor solución es: [34, 33, 20, 35, 36, 31, 17, 1, 6, 4, 3, 2, 27, 28, 29, 30, 38, 22, 39, 24, 40, 21, 9, 23, 41, 8, 10, 25, 11, 12, 18, 26, 5, 13, 19, 14, 16, 15, 37, 7, 0, 32] 
con Fitness  1324

La mejor solución es: [37, 17, 31, 36, 35, 20, 33, 34, 32, 27, 2, 28, 29, 30, 38, 22, 39, 24, 40, 21, 9, 23, 41, 8, 10, 25, 11, 12, 18, 26, 13, 19, 5, 6, 4, 3, 0, 1, 7, 14, 16, 15] 
con Fitness  1317 

La mejor solución es: [1, 7, 14, 16, 15, 37, 17, 31, 36, 35, 20, 33, 34, 32, 28, 29, 30, 38, 22, 39, 21, 24, 40, 23, 41, 9, 8, 10, 25, 11, 12, 18, 26, 13, 19, 5, 6, 4, 3, 2, 27, 0] 
con Fitness  1315 

La mejor solución es: [9, 21, 40, 24, 39, 22, 38, 34, 33, 20, 35, 36, 31, 17, 7, 37, 15, 16, 14, 19, 13, 5, 26, 6, 1, 0, 32, 30, 29, 28, 2, 27, 3, 4, 18, 12, 11, 25, 10, 8, 41, 23] 
con Fitness  1313 
